In [2]:
# Schritt 1: Bibliotheken importieren
import os
import pandas as pd
import plotly.express as px
#from jupyter_dash import JupyterDash
from dash import Dash
from dash import dcc, html
from dash.dependencies import Input, Output

# Schritt 2: CSV-Datei herunterladen (nur wenn sie nicht vorhanden ist)
csv_url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_dash.csv'
csv_filename = 'spacex_launch_dash.csv'

if not os.path.exists(csv_filename):
    import requests
    response = requests.get(csv_url)
    with open(csv_filename, 'wb') as f:
        f.write(response.content)
    print("Datei heruntergeladen:", csv_filename)
else:
    print("Datei bereits vorhanden:", csv_filename)

# Schritt 3: CSV-Datei einlesen
spacex_df = pd.read_csv(csv_filename)
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Schritt 4: Dash-App erstellen
#app = JupyterDash(__name__)
app = Dash(__name__)

app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    
    dcc.Dropdown(
        id='site-dropdown',
        options=[{'label': 'All Sites', 'value': 'ALL'}] +
                [{'label': site, 'value': site} for site in spacex_df['Launch Site'].unique()],
        value='ALL',
        placeholder="Select a Launch Site",
        searchable=True
    ),
    html.Br(),

    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),
    dcc.RangeSlider(
        id='payload-slider',
        min=0,
        max=10000,
        step=1000,
        marks={i: f'{i}' for i in range(0, 10001, 2500)},
        value=[min_payload, max_payload]
    ),
    html.Br(),

    html.Div(dcc.Graph(id='success-payload-scatter-chart'))
])

# Schritt 5: Callback für Pie-Chart
@app.callback(
    Output('success-pie-chart', 'figure'),
    Input('site-dropdown', 'value')
)
def update_pie_chart(selected_site):
    if selected_site == 'ALL':
        fig = px.pie(spacex_df, 
                     names='Launch Site', 
                     values='class', 
                     title='Total Success Launches by Site')
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]
        fig = px.pie(filtered_df,
                     names='class',
                     title=f'Total Success vs Failure for site {selected_site}')
    return fig

# Schritt 6: Callback für Scatter-Chart
@app.callback(
    Output('success-payload-scatter-chart', 'figure'),
    [Input('site-dropdown', 'value'),
     Input('payload-slider', 'value')]
)
def update_scatter_chart(selected_site, payload_range):
    low, high = payload_range
    filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= low) & 
                            (spacex_df['Payload Mass (kg)'] <= high)]

    if selected_site != 'ALL':
        filtered_df = filtered_df[filtered_df['Launch Site'] == selected_site]

    fig = px.scatter(filtered_df,
                     x='Payload Mass (kg)',
                     y='class',
                     color='Booster Version Category',
                     title='Correlation between Payload and Success')
    return fig

# Schritt 7: App im Notebook starten
#app.run(debug=True)
app.run(debug=True, use_reloader=False, open_browser=True)

"""
Man kann den Port sezten und dann in einem getrennte Fenster laufen lassen. 
app.run(debug=True, port=8051)
... "http://127.0.0.1:8050" als Adresse eigeben
"""

Datei bereits vorhanden: spacex_launch_dash.csv


'\nMan kann den Port sezten und dann in einem getrennte Fenster laufen lassen. \napp.run(debug=True, port=8051)\n... "http://127.0.0.1:8050" als Adresse eigeben\n'